<a href="https://colab.research.google.com/github/shantanu2383/BALL_2015/blob/main/Accruals_in_the_Cross_Section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=a1684c6046f099b77a5509e2450f484265e59cd8320e9a6d9034623902237c99
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


Dataload

In [ ]:
#CRSP
filepath="/content/gdrive/MyDrive/Accruals/"

file="CRSPMONTHLY.csv"
crsp=pd.read_csv(filepath + file)

crsp['date'] = pd.to_datetime(crsp['date'], format="%Y%m%d")


crsp=crsp[("1960-01-01"<=crsp['date']) & (crsp['date']< "2019-01-01")]
crsp_parsed=crsp[['PERMNO', 'date', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD', 'VOL']]
crsp=crsp_parsed


x=['PERMNO', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']

for i in x:
  crsp[i]=pd.to_numeric(crsp[i], errors='coerce')


#convert returns to percent:
crsp['RET']*=100
crsp['DLRET']*=100


#add market cap column
crsp['mkt_cap']=abs(crsp['SHROUT'] * crsp['ALTPRC'])/1000
crsp['mkt_cap'].replace(0, np.NaN, inplace=True)


#ONLY KEEP US BASED COMMON STOCKS
crsp=crsp[(crsp['SHRCD']==10)|(crsp['SHRCD']==11)]

#rename variables to lower case
crsp= crsp.rename(columns=str.lower)

#filter for relevant exchanges
exchange_mapping = {
    1: 'NYSE', 31: 'NYSE',
    2: 'AMEX', 32: 'AMEX',
    3: 'NASDAQ', 33: 'NASDAQ'
}
#create dictionary to map values of the 'exchcd' column
crsp['exchange'] = np.select(
    [crsp['exchcd'].isin(exchange_mapping.keys()), crsp['exchcd'].notnull()],
    [crsp['exchcd'].map(exchange_mapping), "Other"],
    default='Other')


#Adjust Returns
crsp['ret_adj'] = crsp['ret']

mask1 = pd.isnull(crsp['dlstcd'])
crsp.loc[mask1, 'ret_adj'] = crsp.loc[mask1, 'ret']

mask2 = pd.notnull(crsp['dlstcd']) & pd.notnull(crsp['dlret'])
crsp.loc[mask2, 'ret_adj'] = crsp.loc[mask2, 'dlret']

mask3 = ((551 <= crsp['dlstcd']) & (crsp['dlstcd'] <= 574)) | (crsp['dlstcd'].isin([500, 520, 580, 584]))
crsp.loc[mask3, 'ret_adj'] = -30

crsp.loc[~(mask1 | mask2 | mask3), 'ret_adj'] = -100

crsp.drop(['shrcd', 'dlret', 'dlstcd'], inplace=True, axis=1)

#exclude financial firms 
crsp=crsp[(crsp['siccd']<6000) |(crsp['siccd']>6999) ]


#Add reference date for matching with Compustat

crsp['date']=pd.to_datetime(crsp['date'])
crsp['year']=crsp['date'].dt.year

crsp['reference_date']=pd.to_datetime(crsp['year'].astype(str) +'-06-01')

crsp.loc[crsp['date'].dt.month < 6, 'year'] -= 1

crsp.loc[crsp['date'].dt.month < 6, 'reference_date'] = pd.to_datetime(
    crsp['year'].astype(str) + '-06-01')

crsp = crsp.drop('year', axis=1)

crsp.set_index('date', inplace=True)

crsp.sort_values(by=['date', 'permno'], inplace=True)

<ipython-input-2-384bb7bc77a5>:5: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp=pd.read_csv(filepath + file)
<ipython-input-2-384bb7bc77a5>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['date']=pd.to_datetime(crsp['date'])


In [ ]:
#COMPUSTAT
filepath="/content/gdrive/MyDrive/Accruals/"

compustat=pd.read_csv(filepath + "compustat.csv")

#Operating Profitability
#REVT: Revenue
#COGS: Cost of goods sold
#XSGA-XRD: Reported sales, general and administrative expenses

#Cash Based Operating Profitability
#RECT: Accounts Recievable
#INVT: Inventory
#XPP: Prepaid Expenses
#DRC + DRLT: Deferred Revenue
#AP: Trade Accounts Payable
#XACC: Accrued Expenses

#CBOP-Alternate (Table 3)
#RECCH: Decrease in accounts receivable
#INVCH: Decrease in Inventory
#APALCH: Increase in accounts payable and accrued liabilities

#Accruals
#ACT: Current Assets
#CH: Cash
#LCT: Current Liabilities
#DLC: Debt in Current Liabilities
#TXP: Income Taxes

#Accruals-Alternate 

#RECCH: Decrease in accounts receivable
#INVCH: Decrease in Inventory
#APALCH: Increase in accounts payable and accrued liabilities
#AOLOCH: Net change in other assets and liabilities
#TXACH: Increase in accrued income taxes

<ipython-input-3-6ba830e62b2c>:4: DtypeWarning: Columns (2,15,17,18,19,23,24,29,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat=pd.read_csv(filepath + "compustat.csv")


In [ ]:
compustat= compustat.rename(columns=str.lower)
x=['gvkey', 'lpermno', 'datadate', 'linktype', 'linkenddt', 'seq', 'ceq',
                    'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk', 'indfmt', 'datafmt',
                     'revt', 'cogs', 'xint', 'xsga', 'xrd', 'rect', 'invt', 'xpp', 'drc', 'drlt', 'ap', 'xacc',
                      'cstk', 'caps', 'tstk', 're', 'acominc', 'mkvalt', 'act', 'ch', 'lct', 'dlc', 'txp', 'dp', 'gp',
                        'recch', 'invch', 'apalch', 'aoloch', 'txach']

compustat=compustat[x]


dates=['datadate', 'linkenddt']
for date in dates:
  compustat[date]=pd.to_datetime(compustat[date], format="%Y%m%d", errors='coerce')

#filter columns
compustat['datafmt']=compustat['datafmt'].apply(str)
compustat=compustat[compustat["indfmt"]=="INDL"]
compustat=compustat[compustat['datafmt']=="STD"]

#Only keep valid links
compustat=compustat[(compustat["linktype"]=="LU")| (compustat["linktype"]=="LC")]

#Only keep links active at datadate
compustat=compustat[(compustat['datadate']<=compustat["linkenddt"]) | pd.isnull(compustat['linkenddt'])]


In [ ]:
#calculate book value using FF definition
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT *,
        COALESCE(seq, ceq+pstk, at-lt) + COALESCE(txditc, txdb + itcb, 0)- COALESCE(pstkrv, pstkl, pstk, 0) as book_value
        FROM compustat'''

compustat_calc=pd.read_sql(query, conn)

compustat=compustat_calc

#add reference date for matching- We lag accounting information by 6 months,
#therefore if a firms financial year ends in June, we assume this information is
#available to investors the following June
compustat['datadate']=pd.to_datetime(compustat['datadate'], errors='coerce')

compustat['year']=compustat['datadate'].dt.year
compustat['year']=pd.to_numeric(compustat['year'], errors='coerce')
compustat['reference_date']=compustat['year']+1
compustat['reference_date']=compustat['reference_date'].apply(str)
compustat['reference_date']+="-06-01"
compustat['reference_date']=pd.to_datetime(compustat['reference_date'])

In [ ]:
compustat.sort_values(by=['lpermno', 'datadate'], inplace=True)

#Operating profitability calculation

compustat['op']=compustat['revt']-compustat['cogs']-(compustat['xsga']-compustat['xrd'])

#fill missing values with 0's
for x in ['rect', 'invt', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'act', 'ch', 'lct', 'dlc', 'txp', 'dp']:
  compustat[x]=compustat[x].fillna(0)


#get lag of variables to calculate changes

for x in ['rect', 'invt', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'act', 'ch', 'lct', 'dlc', 'txp', 'dp']:
    compustat[f'{x}_lag1'] = compustat.groupby('lpermno')[x].shift(1)

#calculate deltas
for x in ['rect', 'invt', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'act', 'ch', 'lct', 'dlc', 'txp', 'dp']:
  compustat[f'delta_{x}']=compustat[x]-compustat[f'{x}_lag1']


#cash based operating profitability

compustat['delta_dr']=compustat['delta_drc']+compustat['delta_drlt']

compustat['cbop']=compustat['op']-compustat['delta_rect']-compustat['delta_invt']-compustat['delta_xpp']+compustat['delta_dr']+compustat['delta_ap']+compustat['delta_xacc']

#accruals
compustat['accruals']=compustat['delta_act']-compustat['delta_ch']-(compustat['delta_lct']-compustat['delta_dlc']-compustat['delta_txp'])-compustat['dp']

#cash based operating profitability alternate calculation
compustat['cbop_alternate']=compustat['op']+compustat['recch']+compustat['invch']+compustat['apalch']

#accruals alternate calculation
compustat['accruals_alternate']=-(compustat['recch'])-compustat['invch']-compustat['apalch']-compustat['aoloch']-compustat['txach']

In [ ]:
compustat.rename(columns={'lpermno':'permno'}, inplace=True)
#compustat.drop('index', axis=1, inplace=True)

#compustat.rename(columns={'da_a/t-1': 'da_a_t_1'}, inplace=True)
#crsp.drop(['level_0'],axis=1, inplace=True)
crsp.to_sql("crsp", conn, if_exists='replace')
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, c.gvkey, c.seq, c.ceq, c.at, c.lt, 
        c.txditc, c.txdb, c.itcb, c.pstkrv, c.pstkl, c.pstk,
        c.revt, c.cogs, c.xint, c.xsga, c.xrd, c.rect, c.invt,
        c.xpp, c.drc, c.drlt, c.ap, c.xacc, c.cstk, c.caps, c.tstk,
        c.re, c.acominc, c.mkvalt, c.book_value, c.op, c.cbop, c.accruals, c.gp, c.cbop_alternate, c.accruals_alternate
        FROM crsp
        LEFT JOIN compustat c
        ON crsp.permno=c.permno
        AND crsp.reference_date=c.reference_date
        '''
crsp_comp=pd.read_sql(query, conn)
crsp=crsp_comp

In [ ]:
crsp['date'] = pd.to_datetime(crsp['date'])
me=crsp[(crsp['date'].dt.month==12)]
me['reference_date']=(me['date'].dt.year) + 1
me['reference_date']=me['reference_date'].apply(str)
me['reference_date']+="-06-01"
me['reference_date']=pd.to_datetime(me['reference_date'])
#me.drop(['level_0'], axis=1, inplace=True)
me.to_sql('me', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, mkt_cap as market_equity
        FROM me
        '''
me=pd.read_sql(query, conn)


crsp.to_sql('crsp', conn, if_exists='replace')
me.to_sql('me', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, me.market_equity
        FROM crsp
        LEFT JOIN me
        ON crsp.permno=me.permno
        AND crsp.reference_date=me.reference_date
        '''
crsp_me=pd.read_sql(query, conn)

crsp=crsp_me

<ipython-input-10-844af77dffef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me['reference_date']=(me['date'].dt.year) + 1
<ipython-input-10-844af77dffef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me['reference_date']=me['reference_date'].apply(str)
<ipython-input-10-844af77dffef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
#get prior month returns and 2-12 month returns

columns=[]
for x in range(1,13):
  label=f"return_t-{x}"
  columns.append(label)
  crsp[f'return_t-{x}']=crsp.groupby('permno')['ret_adj'].shift(x)


#2-12 month returns
for x in range(2,13):
  crsp[f'return_t-{x}']/=100
  crsp[f'return_t-{x}']+=1

crsp['return_t-2_t-12'] = np.cumprod(crsp.loc[:, 'return_t-2':'return_t-12'], axis=1).iloc[:, -1]

# Subtract 1 and convert the result back to percentage returns
crsp['return_t-2_t-12'] = (crsp['return_t-2_t-12'] - 1) * 100

**Sample Filters**

In [ ]:
crsp['ret_adj'].isnull().sum()

#filter sample
crsp.dropna(subset=['market_equity'], inplace=True)
crsp.dropna(subset=['book_value'], inplace=True)
crsp.dropna(subset=['gp'], inplace=True)
crsp.dropna(subset=['ret_adj'], inplace=True)

#date filter
crsp['date'] = pd.to_datetime(crsp['date'])  # Convert the 'date' column to Pandas Timestamp objects
crsp = crsp[(crsp['date'] >= pd.to_datetime('1963-07-01')) & (crsp['date'] <= pd.to_datetime('2014-12-31'))]

**Construct Variables**

In [ ]:
#scale variables by book value of total assets

crsp['op_scaled']=crsp['op']/crsp['at']
crsp['cbop_scaled']=crsp['cbop']/crsp['at']
crsp['accruals_scaled']=crsp['accruals']/crsp['at']

crsp['cbop_alternate_scaled']=crsp['cbop_alternate']/crsp['at']
crsp['accruals_alternate_scaled']=crsp['accruals_alternate']/crsp['at']

#book value variables
crsp['book_to_market']=crsp['book_value']/crsp['market_equity']
import numpy as np
epsilon = 1e-8  # A small positive value to add to 'book_to_market' to avoid taking the logarithm of non-positive values
crsp['log_book_to_market'] = np.log(crsp['book_to_market'] + epsilon)
crsp['log_market_equity']=np.log(crsp['market_equity']+ epsilon)

#return variables
crsp['return_t-1']/=100
crsp['return_t-2_t-12']/=100

crsp.dropna(subset=['return_t-2_t-12'], inplace=True)



# Table 5

**Fama French Size Sort**

In [ ]:
crsp['date']=pd.to_datetime(crsp['date'])
sb=crsp[(crsp['date'].dt.month==6) & (crsp['exchange']=="NYSE")]
sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, mkt_cap as ff_mkt_equity
        FROM sb
        '''

sb=pd.read_sql(query, conn)


In [ ]:
sb['median_mkt_cap'] = sb.groupby('reference_date')['ff_mkt_equity'].transform('median')

sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, median_mkt_cap
        FROM SB
        '''
sb=pd.read_sql(query, conn)


In [ ]:


crsp.to_sql('crsp', conn, if_exists='replace')
sb.to_sql('sb', conn, if_exists='replace')
query='''
        SELECT DISTINCT crsp.*, sb.median_mkt_cap
        FROM crsp
        LEFT JOIN sb
        ON crsp.reference_date=sb.reference_date
        '''
crsp_sb=pd.read_sql(query, conn)
crsp=crsp_sb
crsp.dropna(subset=['median_mkt_cap'], inplace=True)

crsp['size_portfolio']=np.where(crsp['market_equity']>crsp['median_mkt_cap'], "B", "S")



In [ ]:
crsp['date']=pd.to_datetime(crsp['date'])
bb=crsp[crsp['exchange']=="NYSE"]
bb.drop('level_0', axis=1, inplace=True)
bb.to_sql('bb', conn, if_exists='replace')

query='''SELECT DISTINCT permno, reference_date, op_scaled, cbop_scaled, accruals_scaled
          FROM bb
          '''

bb=pd.read_sql(query, conn)

In [ ]:
bb['op_30'] = bb.groupby('reference_date')['op_scaled'].transform(lambda x: x.quantile(0.3))
bb['op_70'] = bb.groupby('reference_date')['op_scaled'].transform(lambda x: x.quantile(0.7))

bb['cbop_30'] = bb.groupby('reference_date')['cbop_scaled'].transform(lambda x: x.quantile(0.3))
bb['cbop_70'] = bb.groupby('reference_date')['cbop_scaled'].transform(lambda x: x.quantile(0.7))


bb['accruals_30'] = bb.groupby('reference_date')['accruals_scaled'].transform(lambda x: x.quantile(0.3))
bb['accruals_70'] = bb.groupby('reference_date')['accruals_scaled'].transform(lambda x: x.quantile(0.7))


In [ ]:
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, op_30, op_70, cbop_30, cbop_70, accruals_30, accruals_70
        FROM bb
        '''

bb=pd.read_sql(query, conn)



In [ ]:
#crsp.drop('level_0', axis=1, inplace=True)

crsp.to_sql('crsp', conn, if_exists='replace')
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, bb.op_30, bb.op_70, bb.cbop_30, bb.cbop_70, bb.accruals_30, bb.accruals_70
        FROM crsp
        LEFT JOIN bb
        ON crsp.reference_date=bb.reference_date
        '''
crsp_bb=pd.read_sql(query, conn)
crsp=crsp_bb

In [ ]:
crsp['op_portfolio'] = np.where(crsp['op_scaled'] < crsp['op_30'], 'weak', 
                               np.where(crsp['op_scaled'] > crsp['op_70'], 'robust', 'neutral'))


crsp['cbop_portfolio'] = np.where(crsp['cbop_scaled'] < crsp['cbop_30'], 'weak', 
                               np.where(crsp['cbop_scaled'] > crsp['cbop_70'], 'robust', 'neutral'))

crsp['accruals_portfolio'] = np.where(crsp['accruals_scaled'] < crsp['accruals_30'], 'weak', 
                               np.where(crsp['accruals_scaled'] > crsp['accruals_70'], 'robust', 'neutral'))


In [ ]:
row_names = ['Average Annualized Return', 'Standard Deviation', 't-value']
column_names = ['Mkt-RF', 'SMB', 'HML', 'UMD', 'ACC', 'RMW_OP', 'RMW_CBOP']

table_5= pd.DataFrame(index=row_names, columns=column_names)

In [ ]:
for x,y in zip(['op', 'cbop', 'accruals'], ['RMW_OP', 'RMW_CBOP', 'ACC']):
  returns=crsp.groupby(['date', 'size_portfolio', f'{x}_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret_adj']), weights=pd.to_numeric(x['market_equity'])))
  returns=returns.to_frame().reset_index()
  returns = returns.pivot(index='date', columns=['size_portfolio', f'{x}_portfolio'], values=0)

  returns.columns = returns.columns.map('_'.join)
  returns.reset_index(inplace=True)

  if x=='accruals':
    returns['HML'] = 0.5 * (returns['S_weak'] + returns['B_weak']) - 0.5 * (returns['S_robust'] + returns['B_robust'])
  else:
    returns['HML'] = 0.5 * (returns['S_robust'] + returns['B_robust']) - 0.5 * (returns['S_weak'] + returns['B_weak'])

  hml=returns['HML'].mean()
  hml/=100
  annualized_average_return = ((1 + hml) ** 12) - 1
  annualized_average_return=annualized_average_return*100

  # Calculate the monthly decimal returns
  returns['HML_decimal'] = returns['HML'] / 100 + 1

  # Calculate the standard deviation of the monthly decimal returns
  monthly_std_decimal = returns['HML_decimal'].std()

  # Annualize the standard deviation
  annualized_std_decimal = (monthly_std_decimal * np.sqrt(12))*100

  import numpy as np

  # Calculate the mean and standard deviation of HML
  hml_mean = returns['HML'].mean()
  hml_std = returns['HML'].std()

  # Calculate the number of observations (N)
  N = len(returns)

  # Calculate the standard error of the mean
  hml_standard_error = hml_std / np.sqrt(N)

  # Calculate the t-statistic
  hml_t_statistic = hml_mean / hml_standard_error

  table_5.loc['Average Annualized Return', y] = annualized_average_return
  table_5.loc['Standard Deviation', y] = annualized_std_decimal
  table_5.loc['t-value', y] = hml_t_statistic








In [ ]:
table_5=table_5.round(2)
table_5

,Mkt-RF,SMB,HML,UMD,ACC,RMW_OP,RMW_CBOP
Average Annualized Return,NaN,NaN,NaN,NaN,2.25494,3.637287,5.777456
Standard Deviation,NaN,NaN,NaN,NaN,5.765567,9.508552,8.659726
t-value,NaN,NaN,NaN,NaN,2.753272,2.676286,4.623803


Add in Standard Fama French Factors